In [ ]:
import json
import openai
import os
import re

openai.api_key = 'MY_OPENAI_API_KEY' # replace with your OpenAI API key

In [ ]:
SQL_SCRIPTS_PATH = './sql_scripts' # input SQL scripts path, 1 script per table
JSON_STORE_PATH = './jsons' # path to store a JSON file with results
JSON_NAME = 'my_schema.json' # name of the JSON file
print(os.listdir(SQL_SCRIPTS_PATH))

In [ ]:
def get_completion(prompt, model='gpt-3.5-turbo', temperature=0):
    messages = [{'role': 'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message['content']

In [ ]:
# read a SQL script; keep only the CREATE statement,
# i.e. if there are any INSERT statements, they will be removed; 
# you can specify the encoding to read a file if needed
def read_create_statement_from_sql_script(path, encoding=None):
    with open(path, encoding=encoding) as f:
        create_st = re.split('(insert into)', f.read(), flags=re.IGNORECASE)[0].rstrip()
    return create_st

In [ ]:
# define a task for ChatGPT to convert SQL queries into the JSON format
base_prompt = f'''
Translate the SQL query for table creation below, which is delimited by triple backticks, into JSON format.
Use "table_name" and "columns" as keys. Any other table attributes (such as primary key, constraints, etc.) \
must also be presented as keys. The "columns" field must be an array of records, with each record containing \
"name" and "type" as required keys, and any other optional keys (such as not null, default, auto_increment, etc.). \
If default is NULL, insert null, not string. If default is numeric (int, float, etc.), insert numeric value, not string.
Make your response as short as possible so it can be used as Python dict.
SQL query:
'''

In [ ]:
# convert SQL queries and collect results in a list
json_tables = []
for sql_script in os.listdir(SQL_SCRIPTS_PATH):
    sql_query = read_create_statement_from_sql_script(f'{SQL_SCRIPTS_PATH}/{sql_script}')
    print(f'*** INPUT ***\n{sql_script}:\n{sql_query}')
    response = get_completion(base_prompt + f'```{sql_query}```')
    try:
        json_query = json.loads(response)
    except Exception as e:
        print(e)
        json_query = {
            'sql_query': sql_query, 
            'response': response, 
            'error': e
        }
    json_tables.append(json_query)
    print(f'\n*** OUTPUT ***\nJSON:\n{json_query}')
    print('-' * 100)

In [ ]:
# save the result in the JSON format
with open(f'{JSON_STORE_PATH}/{JSON_NAME}', 'w') as f:
    json.dump(json_tables, f, indent=4)